In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import requests
import json

## 0. Connect Qlik Sense QRS API

In [ ]:
requests.packages.urllib3.disable_warnings()

# necessary headers
xrf = '<XRFKEY>'
headers = {'X-Qlik-xrfkey': xrf,
"Content-Type": "application/json",
"X-Qlik-User":"UserDirectory=<USER_DIRECTORY>;UserId=<QLIK_USER_ID>"}

#certificate path
cert = ("client.pem", "client_key.pem")

In [ ]:
import requests
import json

# Define a function to handle requests and file writing
def fetch_and_write(url, headers, cert, file_name):
    response = requests.get(url, headers=headers, verify=False, cert=cert)
    if response.status_code == 200:
        data = response.json()
        with open(file_name, 'w') as json_file:
            json.dump(data, json_file)
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")

# Base URL and headers
base_url = 'https://<SERVER_ADRESS>:<PORT>/qrs'
xrf_key = xrf
headers = headers
cert = cert

# Define endpoints and corresponding output file names
endpoints = {
    'app/full': 'app.json',
    'task/full': 'task.json',
    'compositeevent/full': 'compositeevent.json' }

# Loop through endpoints and fetch data
for endpoint, file_name in endpoints.items():
    url = f"{base_url}/{endpoint}?xrfkey={xrf_key}"
    fetch_and_write(url, headers, cert, file_name)

## 1. GET Qlik Sense Apps

In [ ]:
# Call the endpoint to get the list of Qlik Sense apps
with open('app.json', 'r') as file:
    apps = json.load(file)

In [ ]:
print("List of app properties: \n" , list(apps[0].keys()))

In [ ]:
# Desired app properties
properties = {'id':[],
           'name':[],
           'published':[],
           'description':[],
           'stream':[],
           'fileSize':[],
           'lastReloadDate':[],
           'lastReloadTime':[],
           'owner':[]}

In [ ]:
# Getting & manipulating necessary properties from JSON to the desired format
for app in apps:
    app["lastReloadDate"] = datetime.strptime(app["lastReloadTime"], '%Y-%m-%dT%H:%M:%S.%fZ').date()
    app["lastReloadTime"] = datetime.strptime(app["lastReloadTime"], '%Y-%m-%dT%H:%M:%S.%fZ').time()
    app["fileSize"] = round(app["fileSize"]/1000000)
    app["owner"] = app["owner"]["name"]
    if app["stream"] == None: # some apps has no stream
        app["stream"] = np.nan
    elif type(app["stream"]) == type({}):
        app["stream"] = app["stream"]["name"]

In [ ]:
# Creating dataframe for apps with desired properties
for key, value in properties.items():
    for app in apps:
        value.append(app[key])
app_df = pd.DataFrame(properties).set_index('id')

In [ ]:
print("Number of apps: ", len(app_df))
print("Number of published apps: ", app_df["published"].sum())
app_df.head()

In [ ]:
print("Total Size of Apps :", app_df.groupby("published").fileSize.sum()/1000)

In [ ]:
print("Number of apps per stream: ", app_df.groupby('stream').name.count())

In [ ]:
app_df.to_excel('app_df.xlsx')

## 2. GET Qlik Sense Tasks

In [ ]:
# Call the endpoint to get the list of Qlik Sense tasks
with open('task.json', 'r') as file:
    tasks = json.load(file)

In [ ]:
print("List of task properties: \n" , list(tasks[0].keys()))

In [ ]:
# Desired task properties
properties = {'id':[],
              'name':[],
              'enabled':[],
              'nextExecutionDate':[],
              'duration':[],
              'app_id':[]}

In [ ]:
# Getting & manipulating necessary properties from JSON to the desired format
for task in tasks:
    task["nextExecutionDate"] = datetime.strptime(task["operational"]["nextExecution"], '%Y-%m-%dT%H:%M:%S.%fZ')
    #task["duration"] = round(task["operational"]["lastExecutionResult"]["duration"]/(1000*60))
    task["duration"] = timedelta(milliseconds=task["operational"]["lastExecutionResult"]["duration"])
    if task["app"] == None:
        task["app_id"] = ''
    elif type(task["app"]) == type({}):
        task["app_id"] = task["app"]["id"]

In [ ]:
# Creating dataframe for tasks with desired properties
for key, value in properties.items():
    for task in tasks:
        value.append(task[key])

In [ ]:
task_df = pd.DataFrame(properties).set_index('id')
print("Number of tasks: ", len(task_df))
print("Number of enabled tasks: ", task_df["enabled"].sum())
task_df.head()

## 3. GET Qlik Sense Task Triggers (Schemaevent)

In [ ]:
# Call the endpoint to get the list of Qlik Sense schemaevents
with open('schemaevent.json', 'r') as file:
    schemaevents = json.load(file)

In [ ]:
print("List of schemaevent properties: \n" , list(schemaevents[0].keys()))

In [ ]:
# Desired schemaevent properties
properties = {'id':[],
              'schemaFilterDescription':[],
              'incrementDescription':[],
              'enabled':[],
              'reloadTask_id':[],
              'startDate':[]}

In [ ]:
for schemaevent in schemaevents:
    schemaevent["reloadTask_id"] = schemaevent["reloadTask"]["id"]
    schemaevent["startDate"] = (datetime.strptime(schemaevent["startDate"], '%Y-%m-%dT%H:%M:%S.%f'))

In [ ]:
# Creating dataframe for schemaevents with desired properties
for key, value in properties.items():
    for schemaevent in schemaevents:
        value.append(schemaevent[key])

In [ ]:
schemaevent_df = pd.DataFrame(properties).set_index('id')
schemaevent_df.head()

In [ ]:
# schemaFilterDescription has parameters of Trigger Start on Schedule
# The parameters should be converted to period 
schemaevent_df["schemaFilterDescription"] = schemaevent_df["schemaFilterDescription"].copy().apply(lambda x: str(x[0]))
schedule_parameters = schemaevent_df['schemaFilterDescription'].str.split(' ', expand=True)
schedule_parameters.head()

In [ ]:
def parameter_to_period(x):
    # Check if the fourth element in the list(=weekday) is not '*'
    if x[3] != '*':
        return 'weekly'
    # Check if the sixth element in the list(=monthday) is not '*'
    elif x[5] != '*':
        return 'monthly'
    # Check if the "incrementDescription" key in the dictionary has the value '0 1 0 0'
    elif x["incrementDescription"] == '0 1 0 0':
        return 'hourly'
    else:
        return 'daily'

In [ ]:
# schedule_parameters merged to schemaevent df
schemaevent_df2 = schemaevent_df.merge(schedule_parameters, how='left', left_index=True, right_index=True)

In [ ]:
# schedule parameters converted to period
period = schemaevent_df2.apply(parameter_to_period, axis=1)
schemaevent_df2["period"] = period

In [ ]:
# New dataframe with desired properties
trigger = schemaevent_df2.copy()[["reloadTask_id","enabled", "startDate", "period"]]
print("Number of triggers: ", len(trigger))
trigger.head()

In [ ]:
# adding period properties 
trigger["hour"] = schemaevent_df2.copy().loc[:,"incrementDescription"].str.split(' ', expand=True)[1]
trigger["weekday"] = schemaevent_df2.loc[:,3]
trigger["monthday"] = schemaevent_df2.loc[:,5]

In [ ]:
trigger.sort_values(by="startDate", inplace=True) 
trigger = trigger.set_index("reloadTask_id")
trigger.head()

### 3.1 GET Event Task Triggers (compositeevent)

In [ ]:
# Call the endpoint to get the list of Qlik Sense compositeevent(Task Triggers)
with open('compositeevent.json', 'r') as file:
    compositeevents = json.load(file)

In [ ]:
print("List of compositeevent properties: \n" , list(schemaevents[0].keys()))

In [ ]:
# Desired compositeevent properties
properties = {'id':[],
              'reloadTask_id':[],
              'enabled':[],
              'preceeding_reloadTask_id':[]}

In [ ]:
# Getting & manipulating necessary properties from JSON to the desired format
for compositeevent in compositeevents:
    compositeevent["reloadTask_id"] = compositeevent["reloadTask"]["id"]
    compositeevent["preceeding_reloadTask_id"] = compositeevent["compositeRules"][0]["reloadTask"]["id"]

In [ ]:
for key, value in properties.items():
    for compositeevent in compositeevents:
        value.append(compositeevent[key])

In [ ]:
compositeevent_df = pd.DataFrame(properties).set_index('id')
compositeevent_df = compositeevent_df.set_index("reloadTask_id")
print("Number of task triggers: ", len(compositeevent_df))
compositeevent_df.head()

In [ ]:
# Concating Scheduled & Task Triggers
trigger_all = pd.concat([trigger, compositeevent_df])
print("Number of triggers: ", len(trigger_all))
trigger_all.head()

## 4. Creating Schedule Dataframe

In [ ]:
task_app_df = task_df.merge(app_df, how='left', left_on='app_id', right_index=True, suffixes=('', '_app'))

In [ ]:
task_app_df.head(2)

In [ ]:
task_app_trigger_df = task_app_df.merge(trigger_all, how='inner', left_index=True, right_index=True, suffixes=('', '_trigger'))

In [ ]:
print("Number of tasks: ", len(task_app_trigger_df))
#df2.head()

In [ ]:
task_app_trigger_df.head(2)

In [ ]:
task_app_trigger_df.loc["77b7bea3-1f70-4d9b-839a-37d1076d9dd9", "stream"] = 'QVD' #correction

## 5. Filtering for Enabled Tasks & Published Apps

In [ ]:
task_schedule = task_app_trigger_df[
    (task_app_trigger_df["enabled"] != False) & \
    (task_app_trigger_df["published"] != False) & \
    (task_app_trigger_df["enabled_trigger"] != False) & \
    (~task_app_trigger_df["stream"].isin(["Kullanılmayan Raporlar", "Archive"]))]
len(task_schedule)

In [ ]:
task_schedule = task_schedule.copy()
task_schedule["endDate"] = task_schedule["startDate"] + task_schedule["duration"]
task_schedule.loc[:,"duration_min"] = task_schedule["duration"].apply(lambda x: round(x.total_seconds() / 60 + 0.5))

In [ ]:
task_schedule.head(2)

In [ ]:
task_schedule.to_excel('task_schedule.xlsx')

## 6. Getting Start and End time for task triggers

In [ ]:
columns = ['name',
           'startDate',
           'endDate',
           'duration',
           'duration_min',
           'period',
           'hour',
           'weekday',
           'monthday',
           'preceeding_reloadTask_id']

In [ ]:
schedule = task_schedule[columns].copy()

In [ ]:
schedule.head(2)

In [ ]:
schedule.info()

In [ ]:
for i in range (10):
    startDate = []
    endDate = []
    period = []
    for index, row in schedule.iterrows():
        if pd.isna(row["preceeding_reloadTask_id"]):
            startDate.append(row["startDate"])
            endDate.append(row["endDate"])
            period.append(row["period"])
        else:
            preceding_id = row["preceeding_reloadTask_id"]
            preceding_end_date = schedule.loc[preceding_id, "endDate"]
            periods = schedule.loc[preceding_id, "period"]
            if isinstance(preceding_end_date, pd.Series):
                startDate.append(preceding_end_date.values[0])
                endDate.append(preceding_end_date.values[0] + row["duration"])
                period.append(periods.values[0])
            elif pd.isna(preceding_end_date):  # Correct condition
                startDate.append(np.nan)
                endDate.append(np.nan)
                period.append(np.nan)
            else:
                startDate.append(preceding_end_date)
                endDate.append(preceding_end_date + row["duration"])
                period.append(periods)
    schedule["startDate"] = startDate              
    schedule["endDate"] = endDate
    schedule["period"] = period

In [ ]:
schedule["startTime"] = schedule["startDate"].apply(lambda x: x.replace(second=0, microsecond=0).time())
schedule["endTime"] = schedule["endDate"].apply(lambda x: x.replace(second=0, microsecond=0).time())

In [ ]:
daily_schedule = schedule[schedule["period"] == 'daily'][["startTime","endTime","duration"]]
daily_schedule = daily_schedule.sort_values("startTime")

In [ ]:
daily_schedule.head()

## 7. Creating a Gantt Chart for tasks

In [ ]:
# Filter tasks within the 05:00 to 14:00 range
start_range = time(3, 0)
end_range = time(12, 0)

filtered_df = daily_schedule[(daily_schedule['startTime'] >= start_range) & (daily_schedule['startTime'] <= end_range)]
filtered_df = filtered_df.reset_index()

In [ ]:
# Convert times back to datetime for plotting
filtered_df['startTime'] = filtered_df['startTime'].apply(lambda t: datetime.combine(datetime.today(), t))
filtered_df['endTime'] = filtered_df['endTime'].apply(lambda t: datetime.combine(datetime.today(), t))

In [ ]:
# Plotting the Gantt chart
fig, ax = plt.subplots(figsize=(10, 7))

for i, row in filtered_df.iterrows():
    ax.plot([row['startTime'], row['endTime']], [i, i], color='blue', linewidth=3)

# Formatting the x-axis to show time in the range of 05:00 - 14:00
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlim([datetime.combine(datetime.today(), start_range), datetime.combine(datetime.today(), end_range)])
ax.set_ylim(-1, len(filtered_df))
ax.set_yticks(range(len(filtered_df)))
ax.set_yticklabels(filtered_df.index)

plt.xlabel('Time')
plt.ylabel('Task ID')
plt.title('Gantt Chart for Tasks (08:00 - 10:00)')
plt.grid(True)
plt.show()

## 8. Task Overlap Analysis

In [ ]:
# Create a timeline with one-minute increments
timeline = pd.date_range(start='2024-08-04 06:00:00', end='2024-08-04 12:00:00', freq='min')

In [ ]:
minutes = []
overlap_counts = []
overlapping_tasks = []
for dtime in timeline:
    count = ((filtered_df['startTime'] <= dtime) & (filtered_df['endTime'] > dtime)).sum()
    if count > 6:
        minutes.append(dtime)
        overlap_counts.append(count)
        o_task = list(filtered_df[(filtered_df['startTime'] <= dtime) & (filtered_df['endTime'] > dtime)]["id"].values)
        overlapping_tasks.append(o_task)

In [ ]:
pd.DataFrame(zip(minutes,overlap_counts,overlapping_tasks),columns=['minute','task_count','task_ids'])